In [1]:
# header
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

import pandas as pd
import geopandas as gpd
from siuba import *

pd.set_option('display.max_columns', None) 

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/safety_projects/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [2]:
# load analytical file
crashes_encampments_gdf = gpd.read_parquet(f'{GCS_FILE_PATH}analytical_file_joined.parquet')

# Overall Stats on Analytical File

In [3]:
crashes_encampments_gdf.corr()

/tmp/ipykernel_3411/3142507805.py:1: FutureWarning: The default value of numeric_only in GeoDataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.


,NUMBER_KILLED,NUMBER_INJURED,ped_crash,bike_crash,pedbike_crash,WONO,WO_density,WO_dummy,WO_density_pctile
NUMBER_KILLED,1.000000,0.389100,0.688502,0.267891,0.675144,0.314430,0.270134,0.255537,0.268964
NUMBER_INJURED,0.389100,1.000000,0.795320,0.465364,0.834446,0.263034,0.234166,0.269471,0.281849
ped_crash,0.688502,0.795320,1.000000,0.279087,0.942231,0.370012,0.329423,0.336344,0.352172
bike_crash,0.267891,0.465364,0.279087,1.000000,0.583538,0.161729,0.137012,0.181645,0.191602
pedbike_crash,0.675144,0.834446,0.942231,0.583538,1.000000,0.369486,0.326458,0.347967,0.364837
WONO,0.314430,0.263034,0.370012,0.161729,0.369486,1.000000,0.934967,0.410077,0.469968
WO_density,0.270134,0.234166,0.329423,0.137012,0.326458,0.934967,1.000000,0.468496,0.534078
WO_dummy,0.255537,0.269471,0.336344,0.181645,0.347967,0.410077,0.468496,1.000000,0.992071
WO_density_pctile,0.268964,0.281849,0.352172,0.191602,0.364837,0.469968,0.534078,0.992071,1.000000


In [4]:
crashes_encampments_gdf.groupby('DIST_left')[['ped_crash','pedbike_crash','WO_density','WO_dummy']].corr()

ped_crash  pedbike_crash  WO_density  WO_dummy
DIST_left                                                              
1         ped_crash       1.000000       0.871718    0.179693  0.163949
          pedbike_crash   0.871718       1.000000    0.126853  0.110048
          WO_density      0.179693       0.126853    1.000000  0.781458
          WO_dummy        0.163949       0.110048    0.781458  1.000000
10        ped_crash       1.000000       0.890581    0.424995  0.336628
          pedbike_crash   0.890581       1.000000    0.483028  0.377696
          WO_density      0.424995       0.483028    1.000000  0.779760
          WO_dummy        0.336628       0.377696    0.779760  1.000000
11        ped_crash       1.000000       0.963930    0.506483  0.448448
          pedbike_crash   0.963930       1.000000    0.519841  0.466067
          WO_density      0.506483       0.519841    1.000000  0.553170
          WO_dummy        0.448448       0.466067    0.553170  1.000000
12        ped_crash       1.000000       0.923446    0.115851  0.172648
          pedbike_crash   0.923446       1.000000    0.127402  0.182636
          WO_density      0.115851       0.127402    1.000000  0.816278
          WO_dummy        0.172648       0.182636    0.816278  1.000000
2         ped_crash       1.000000       0.949732    0.239135  0.217858
          pedbike_crash   0.949732       1.000000    0.266398  0.248043
          WO_density      0.239135       0.266398    1.000000  0.872358
          WO_dummy        0.217858       0.248043    0.872358  1.000000
3         ped_crash       1.000000       0.938386    0.538859  0.432887
          pedbike_crash   0.938386       1.000000    0.584371  0.466119
          WO_density      0.538859       0.584371    1.000000  0.696702
          WO_dummy        0.432887       0.466119    0.696702  1.000000
4         ped_crash       1.000000       0.914939    0.182847  0.236404
          pedbike_crash   0.914939       1.000000    0.203739  0.253562
          WO_density      0.182847       0.203739    1.000000  0.727240
          WO_dummy        0.236404       0.253562    0.727240  1.000000
5         ped_crash       1.000000       0.776427    0.234161  0.272434
          pedbike_crash   0.776427       1.000000    0.254855  0.293988
          WO_density      0.234161       0.254855    1.000000  0.789676
          WO_dummy        0.272434       0.293988    0.789676  1.000000
6         ped_crash       1.000000       0.923945    0.255226  0.204741
          pedbike_crash   0.923945       1.000000    0.270881  0.203860
          WO_density      0.255226       0.270881    1.000000  0.615619
          WO_dummy        0.204741       0.203860    0.615619  1.000000
7         ped_crash       1.000000       0.962071    0.390540  0.344417
          pedbike_crash   0.962071       1.000000    0.385262  0.346261
          WO_density      0.390540       0.385262    1.000000  0.405088
          WO_dummy        0.344417       0.346261    0.405088  1.000000
8         ped_crash       1.000000       0.958511    0.127981  0.240927
          pedbike_crash   0.958511       1.000000    0.121275  0.233681
          WO_density      0.127981       0.121275    1.000000  0.589535
          WO_dummy        0.240927       0.233681    0.589535  1.000000
9         ped_crash            NaN            NaN         NaN       NaN
          pedbike_crash        NaN       1.000000   -0.008264 -0.008264
          WO_density           NaN      -0.008264    1.000000  1.000000
          WO_dummy             NaN      -0.008264    1.000000  1.000000

In [ ]:
# try simple regression
import statsmodels.api as sm

def reg_wo_crashes(yvar, xvar):
    y = crashes_encampments_gdf[yvar]
    X = crashes_encampments_gdf[xvar]
    X = sm.add_constant(X)
    model = sm.GLM(y, X, family=sm.families.Poisson())
    result = model.fit()
    print(result.summary())
    
    return result

In [ ]:
reg_wo_crashes('pedbike_crash', 'WO_dummy')

In [ ]:
# interpret the dummy variable coefficient
import math
math.exp(1.6199)

A bridge associated with an encampment has 5x more pedestrian and bike crashes than a bridge without. 

In [ ]:
reg_wo_crashes('pedbike_crash', 'WO_density')

In [5]:
# take log of pedestrian accident
import numpy as np

crashes_encampments_gdf['pedbike_crash_log'] = (np.log(crashes_encampments_gdf['pedbike_crash'])).replace(np.NINF, 0) 
crashes_encampments_gdf['ped_crash_log'] = (np.log(crashes_encampments_gdf['ped_crash'])).replace(np.NINF, 0) 

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log


In [6]:
# try adding district (mixed-effect model, variable effect for district)
import statsmodels.formula.api as smf

model = smf.mixedlm(formula='pedbike_crash_log ~ WO_dummy', data=crashes_encampments_gdf, groups = crashes_encampments_gdf["DIST_left"], missing='drop')
result = model.fit()
print(result.summary())

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: pedbike_crash_log
No. Observations: 8972    Method:             REML             
No. Groups:       12      Scale:              0.0639           
Min. group size:  122     Log-Likelihood:     -413.5577        
Max. group size:  1659    Converged:          Yes              
Mean group size:  747.7                                        
-----------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|   [0.025   0.975]
-----------------------------------------------------------------
Intercept     0.019      0.009    2.037   0.042    0.001    0.037
WO_dummy      0.194      0.007   28.232   0.000    0.181    0.208
Group Var     0.001      0.002                                   



/opt/conda/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.


In [8]:
import math
math.exp(0.194)

1.2140962829562332

After controlling for district, A bridge associated with an encampment has 1.21x more pedestrian or bike crashes than a bridge without.

In [9]:
# try adding district (mixed-effect model, variable effect for district)
model = smf.mixedlm(formula='ped_crash_log ~ WO_dummy', data=crashes_encampments_gdf, groups = crashes_encampments_gdf["DIST_left"], missing='drop')
result = model.fit()
print(result.summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: ped_crash_log
No. Observations: 8972    Method:             REML         
No. Groups:       12      Scale:              0.0463       
Min. group size:  122     Log-Likelihood:     1031.4577    
Max. group size:  1659    Converged:          Yes          
Mean group size:  747.7                                    
------------------------------------------------------------
              Coef.  Std.Err.    z     P>|z|  [0.025  0.975]
------------------------------------------------------------
Intercept     0.012     0.008   1.530  0.126  -0.003   0.028
WO_dummy      0.149     0.006  25.335  0.000   0.137   0.160
Group Var     0.001     0.001                               



/opt/conda/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.


In [10]:
math.exp(0.149)

1.1606729892090855

After controlling for district, A bridge associated with an encampment has 1.16x more pedestrian crashes than a bridge without.

In [ ]:
# export geojson for ArcGIS Pro
shared_utils.utils.geojson_gcs_export(crashes_encampments_gdf.drop("geometry_y", axis=1), GCS_FILE_PATH, "analytical_file_joined")  